In [ ]:
!pip install pyspark
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
import spacy
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3e9958ba21b3416b144287c9473b2ff8c3c604bb4e60478a4e5ee1b9f1e8f5d4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Autenticación
auth.authenticate_user()

# Proyecto y cliente de BigQuery
project_id = 'bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Consulta
query = f"""
    SELECT *
    FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_productos`
    WHERE country = 'PE'
"""

# Cargar el DataFrame
df_spm = client.query(query).to_dataframe()

# Ver las primeras filas del DataFrame
df_spm.head()

,country,sellerId,sellerName,sku_seller,product_name,brand_name,model,shop_sku,primary_category,global_identifier,...,rlo_items_problema_cobro,rlo_items_producto_incompleto,rlo_items_publicidad_enganosa,rlo_items_falla_producto,customer_reviews,customer_reviews_1,customer_reviews_2,customer_reviews_3,customer_reviews_4,customer_reviews_5
0,PE,SCD0DE7,MODA & COLOR,FBFL00508-100-00,Suplemento Colageno Hidrolizado Con Camu Camu ...,ZOHAR,COLAGENO HIDROLIZADO,124738632,Medicamentos farmaceuticos,G1606,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,PE,SCA3D8E,DS VENTURE,SE0649-20-3,Vibrador Rabbit - Echanter Exciter,FOREPLAY,Foreplay,125099838,Medicamentos farmaceuticos,G1606,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,PE,SC66A00,WELL DONE,120015,NUTRAZUL ARNICA SPORT SPRAY FRASCO X 75ML,PHARMARIS,spray,125569598,Medicamentos farmaceuticos,G1606,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,PE,SC63E40,IMPORTACIONES ELODI SAC IMPORTACIONES ELODI,348_DELETED_2023-07-16_19-32-48,100 Guantes Sinteticos Desechables de Nitrilo ...,GENERICO,Guantes de color negro Talla M,121811312,Mobiliario de peluqueria,G1608,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,PE,SC89B3D,ANDES PERU UAT,20221013230_DELETED_2022-11-17_18-16-28,A - Test product bulk test 3,LIMAR,TEST 003,115223874,Armarios|gabinetes para herramientas,G010101,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
print(df_spm.shape)

(1348283, 176)


In [ ]:
# Cargar el conjunto de datos desde el DataFrame de BigQuery
df = df_spm

# Lista de atributos que deseas mantener
atributos_deseados = [
    'country', 'sellerId', 'sellerName', 'sku_seller', 'product_name','shop_sku',
    'primary_category','N1', 'size', 'width_in_cm','length_in_cm', 'height_in_cm',
    'weight_in_kg', 'width_diff', 'length_diff','height_diff', 'weight_diff'
]

# Crear un nuevo DataFrame solo con los atributos deseados y renombrarlo a df_spm
df_spm = df[atributos_deseados]

# Mostrar las primeras filas del nuevo DataFrame
#df_spm.head()


In [ ]:

#Ver cantidad de nulos
#df_spm.isnull().sum()


In [ ]:
# Eliminar todas las filas que tengan nulos
df_spm = df_spm.dropna()

#Ver la cantidad de nulos, post imputación
#df_spm.isnull().sum()

In [ ]:
# Suma de los valores "No" y "Sí" para cada columna
suma_no_si_width_diff = df_spm['width_diff'].value_counts().to_dict()
suma_no_si_length_diff = df_spm['length_diff'].value_counts().to_dict()
suma_no_si_height_diff = df_spm['height_diff'].value_counts().to_dict()
suma_no_si_weight_diff = df_spm['weight_diff'].value_counts().to_dict()

# Mostrar los resultados
#print("Suma de 'No' y 'Sí' en width_diff:", suma_no_si_width_diff)
#print("Suma de 'No' y 'Sí' en length_diff:", suma_no_si_length_diff)
#print("Suma de 'No' y 'Sí' en height_diff:", suma_no_si_height_diff)
#print("Suma de 'No' y 'Sí' en weight_diff:", suma_no_si_weight_diff)

In [ ]:
df_spm['target'] = 'Correcto'  # Inicializar con 'Correcto'

# Actualizar a 'Incorrecto' si hay algún 'Sí' en las columnas mencionadas
condicion_si = (df_spm['width_diff'] == 'Si') | \
               (df_spm['length_diff'] == 'Si') | \
               (df_spm['height_diff'] == 'Si') | \
               (df_spm['weight_diff'] == 'Si')

df_spm.loc[condicion_si, 'target'] = 'Incorrecto'

In [ ]:
# Eliminar Incorrectos
df_spm_correctos = df_spm[df_spm['target'] == 'Correcto']

# Renombrar el DataFrame
df_spm = df_spm_correctos

# Ver las primeras filas del DataFrame resultante
#df_spm.head()

In [ ]:
# Suma de los valores en la columna 'target'
suma_target = df_spm['target'].value_counts()

# Mostrar el resultado
print(suma_target)


Correcto    1156729
Name: target, dtype: int64


In [ ]:
# Lista de atributos que deseas mantener
atributos_deseados = [
    'sellerName','product_name','shop_sku','primary_category','N1', 'size',
    'width_in_cm','length_in_cm', 'height_in_cm','weight_in_kg',
]

# Crear un nuevo DataFrame solo con los atributos deseados y renombrarlo a df_spm
df_spm = df[atributos_deseados]

# Mostrar las primeras filas del nuevo DataFrame
#df_spm.head()

In [ ]:
df_spm = df_spm.dropna()


In [ ]:
#Ver la cantidad de nulos, post imputación
#df_spm.isnull().sum()

In [ ]:
# Convertir las columnas a tipo numérico
df_spm['width_in_cm'] = pd.to_numeric(df_spm['width_in_cm'], errors='coerce')
df_spm['length_in_cm'] = pd.to_numeric(df_spm['length_in_cm'], errors='coerce')
df_spm['height_in_cm'] = pd.to_numeric(df_spm['height_in_cm'], errors='coerce')

# Calcular la columna de volumen en cm3
df_spm['volumen_cm3'] = df_spm['width_in_cm'] * df_spm['length_in_cm'] * df_spm['height_in_cm']

# Reordenar las columnas colocando 'size' al final
column_order = [col for col in df_spm.columns if col != 'size'] + ['size']
df_spm = df_spm[column_order]

# Verificar las primeras filas del DataFrame actualizado
#df_spm.head()

In [ ]:
print(df_spm.shape)

(1262967, 11)


In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, CountVectorizer, IDF
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType


In [ ]:
# Convertir las columnas a tipo numérico con 4 decimales
numeric_columns = ['width_in_cm', 'length_in_cm', 'height_in_cm', 'weight_in_kg', 'volumen_cm3']
for col in numeric_columns:
    df_spm[col] = pd.to_numeric(df_spm[col], errors='coerce').round(4)

# Reordenar las columnas colocando 'size' al final
column_order = [col for col in df_spm.columns if col != 'size'] + ['size']
df_spm = df_spm[column_order]

# Verificar las primeras filas del DataFrame actualizado
#df_spm.head()

In [ ]:
# Crear un esquema para el DataFrame de Spark
schema = StructType([
    StructField("sellerName", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("shop_sku", StringType(), True),
    StructField("primary_category", StringType(), True),
    StructField("N1", StringType(), True),
    StructField("width_in_cm", FloatType(), True),
    StructField("length_in_cm", FloatType(), True),
    StructField("height_in_cm", FloatType(), True),
    StructField("weight_in_kg", FloatType(), True),
    StructField("volumen_cm3", FloatType(), True),
    StructField("size", StringType(), True),
])


In [ ]:
# Inicializar la sesión de Spark
spark = SparkSession.builder.master("local").appName("SPM-SPARK").getOrCreate()

# Crear el DataFrame de Spark a partir del DataFrame de pandas
df_spark = spark.createDataFrame(df_spm, schema=schema)

# Verificar las primeras filas del DataFrame de Spark
#df_spark.show()

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

2024-01-28 15:49:37.953582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 15:49:37.953623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 15:49:37.954883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 15:49:39.042895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 52.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import nltk
from pyspark.sql.types import ArrayType, StringType
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Cargar modelo de procesamiento de lenguaje natural en español
nlp = spacy.load('es_core_news_sm')

# Obtener las stopwords en español
spanish_stop_words = set(nltk.corpus.stopwords.words('spanish'))

# Definir la función UDF para lematizar texto
def lemmatize_text_udf(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Definir las funciones UDF para tokenizar y limpiar texto
def tokenize_and_clean_udf(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in spanish_stop_words]
    return tokens


In [ ]:
# Definir las funciones UDF para Spark
tokenize_and_clean_spark_udf = udf(tokenize_and_clean_udf, ArrayType(StringType()))
lemmatize_text_spark_udf = udf(lemmatize_text_udf, StringType())

In [ ]:
# Aplicar tokenización y limpieza en un solo paso
df_spark = df_spark.withColumn('product_name_tokens', tokenize_and_clean_spark_udf('product_name'))
df_spark = df_spark.withColumn('primary_category_tokens', tokenize_and_clean_spark_udf('primary_category'))
df_spark = df_spark.withColumn('product_name_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'product_name_tokens')))
df_spark = df_spark.withColumn('primary_category_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'primary_category_tokens')))

In [ ]:
# Aplicar lematización
df_spark = df_spark.withColumn('product_name_cleaned', lemmatize_text_spark_udf('product_name_cleaned'))
df_spark = df_spark.withColumn('primary_category_cleaned', lemmatize_text_spark_udf('primary_category_cleaned'))


In [ ]:
# TF-IDF para 'product_name' y 'primary_category' en una única pipeline
vectorizer_name = CountVectorizer(inputCol='product_name_tokens', outputCol='raw_name_counts_name_tfidf')
idf_name = IDF(inputCol='raw_name_counts_name_tfidf', outputCol='name_tfidf')

vectorizer_category = CountVectorizer(inputCol='primary_category_tokens', outputCol='raw_category_counts_category_tfidf')
idf_category = IDF(inputCol='raw_category_counts_category_tfidf', outputCol='category_tfidf')


In [ ]:
text_pipeline = Pipeline(stages=[
    vectorizer_name, idf_name,
    vectorizer_category, idf_category
])

# Verificar las primeras filas del DataFrame de Spark después del procesamiento de texto
#df_spark.show()

In [ ]:
#df_spark.printSchema()

In [ ]:
#rdd_from_df_spark = df_spark.rdd

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Aplicar el pipeline de procesamiento de texto al DataFrame
df_spark = text_pipeline.fit(df_spark).transform(df_spark)

# Reducción de palabras a las dos primeras en las columnas 'product_name_cleaned' y 'primary_category_cleaned'
df_spark = df_spark.withColumn('product_name_cleaned', F.expr("substring_index(product_name_cleaned, ' ', 2)"))
df_spark = df_spark.withColumn('primary_category_cleaned', F.expr("substring_index(primary_category_cleaned, ' ', 2)"))

# Verificar las primeras filas del DataFrame de Spark después del procesamiento de texto
#df_spark.show()


In [ ]:
# Crear un StringIndexer para convertir 'size' a etiquetas numéricas
indexer = StringIndexer(inputCol='size', outputCol='size_indexed')

# Ajustar las columnas de características según tu conjunto de datos, incluyendo las nuevas características de TF-IDF
feature_columns = ['width_in_cm', 'length_in_cm', 'height_in_cm', 'weight_in_kg', 'volumen_cm3']

# Crear un VectorAssembler para combinar las características en un vector
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Dividir el conjunto de datos en entrenamiento y prueba
train_data, test_data = df_spark.randomSplit([0.7, 0.3], seed=42)

# Modelo de clasificación
classifier = DecisionTreeClassifier(
    labelCol='size_indexed',  # Usar la columna size_indexed como etiqueta
    featuresCol='features',
    maxDepth=28,
    minInstancesPerNode=1
)

# Crear el Pipeline con todas las etapas
pipeline = Pipeline(stages=[
    indexer,
    vector_assembler,
    classifier
])


In [ ]:
# Ajustar el modelo al conjunto de entrenamiento
model = pipeline.fit(train_data)

In [ ]:
# Predicciones sobre los datos de prueba.
predictions = model.transform(test_data)

# Visualizar predicciones
predictions.select('size', 'size_indexed', 'prediction', 'probability').show()

# Crear un evaluador
evaluator = MulticlassClassificationEvaluator(labelCol='size_indexed', predictionCol='prediction', metricName='accuracy')

# Calcular el accuracy
accuracy = evaluator.evaluate(predictions)
print(f'Exactitud del modelo: {accuracy}')


In [ ]:
# Mostrar los valores distintos de size, label y prediction
predictions.select('size', 'size_indexed', 'prediction').distinct().show()


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='size_indexed', predictionCol='prediction', metricName='weightedRecall')
recall = evaluator.evaluate(predictions)
print(f'Recall del modelo: {recall}')


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='size_indexed', predictionCol='prediction', metricName='weightedPrecision')
precision = evaluator.evaluate(predictions)
print(f'Precisión del modelo: {precision}')


In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='size_indexed', predictionCol='prediction', metricName='f1')
f1_score = evaluator.evaluate(predictions)
print(f'F1-Score del modelo: {f1_score}')


In [ ]:
# Ruta local donde se guardará el modelo
local_model_path = "/content/SPM_spark"

# Guardar el modelo en el sistema de archivos local de Colab
model.save(local_model_path)


In [ ]:
from google.colab import files

# Comprimir el modelo para descargarlo como un archivo zip
!zip -r /content/SPM_spark.zip /content/SPM_spark

# Descargar el archivo zip
files.download("/content/SPM_spark.zip")


# --- FINAL DEL MODELO TALLAS ---

# CARGAR EL MODELO Y PROBAR CON UN REGISTRO

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=eb28f6cca8bc74fb5e6f2c8bedb01900a1843f438aeb0854d5119328da1696dd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import zipfile
from pyspark.ml import PipelineModel

In [ ]:
import os

model_directory = "/content/SPM_spark"

# Verificar si el directorio existe
if os.path.exists(model_directory):
    # Listar el contenido del directorio
    content_list = os.listdir(model_directory)

    # Imprimir el contenido del directorio
    print("Contenido del directorio:")
    for item in content_list:
        print(item)

    # Verificar si hay una carpeta 'stages'
    if 'stages' in content_list:
        print("\n¡La carpeta 'stages' existe! Puede contener el modelo.")

    # Verificar si hay una carpeta 'metadata'
    if 'metadata' in content_list:
        print("\n¡La carpeta 'metadata' existe! Puede contener información adicional sobre el modelo.")
else:
    print(f"El directorio {model_directory} no existe.")


Contenido del directorio:
stages
metadata

¡La carpeta 'stages' existe! Puede contener el modelo.

¡La carpeta 'metadata' existe! Puede contener información adicional sobre el modelo.


In [ ]:
#Extraerel zip
# Nombre del archivo zip que subiste
zip_filename = 'SPM_spark.zip'

# Descomprimir el archivo
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('SPM_spark')

In [ ]:
#Cargar el modelo
# Cargar el modelo desde el directorio descomprimido
loaded_model = PipelineModel.load("/content/SPM_spark")

# Ahora puedes utilizar el modelo cargado en Colab


In [ ]:
nuevo_registro = {
    'sellerName': 'FARMATODO Shop',
    'product_name': 'Dolex activgel 24 tabs',
    'shop_sku': '119878204',
    'primary_category': 'Medicamentos farmaceuticos',
    'N1': 'Belleza, cuidado personal, higiene y salud',
    'width_in_cm': 10.0,
    'length_in_cm': 7.0,
    'height_in_cm': 10.0,
    'weight_in_kg': 0.3,
    'volumen_cm3': 700.0,
    'size': 'XS3'
}

In [ ]:
# Importar pandas
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType
from pyspark.sql import functions as F




# Crear un esquema para el DataFrame de Spark
schema = StructType([
    StructField("sellerName", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("shop_sku", StringType(), True),
    StructField("primary_category", StringType(), True),
    StructField("N1", StringType(), True),
    StructField("width_in_cm", FloatType(), True),
    StructField("length_in_cm", FloatType(), True),
    StructField("height_in_cm", FloatType(), True),
    StructField("weight_in_kg", FloatType(), True),
    StructField("volumen_cm3", FloatType(), True),
    StructField("size", StringType(), True),
])

# Inicializar la sesión de Spark
spark = SparkSession.builder.master("local").appName("SPM-SPARK").getOrCreate()


# Definir la función UDF para lematizar texto
def lemmatize_text_udf(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Definir las funciones UDF para tokenizar y limpiar texto
def tokenize_and_clean_udf(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in spanish_stop_words]
    return tokens

# Definir las funciones UDF para Spark
tokenize_and_clean_spark_udf = udf(tokenize_and_clean_udf, ArrayType(StringType()))
lemmatize_text_spark_udf = udf(lemmatize_text_udf, StringType())


In [ ]:
# Crear un DataFrame de pandas con el nuevo registro
nuevo_registro_pd = pd.DataFrame([nuevo_registro])

# Crear un DataFrame de Spark a partir del DataFrame de pandas
nuevo_registro_spark = spark.createDataFrame(nuevo_registro_pd, schema=schema)

# Aplicar tokenización y limpieza en un solo paso al nuevo registro
nuevo_registro_spark = nuevo_registro_spark.withColumn('product_name_tokens', tokenize_and_clean_spark_udf('product_name'))
nuevo_registro_spark = nuevo_registro_spark.withColumn('primary_category_tokens', tokenize_and_clean_spark_udf('primary_category'))
nuevo_registro_spark = nuevo_registro_spark.withColumn('product_name_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'product_name_tokens')))
nuevo_registro_spark = nuevo_registro_spark.withColumn('primary_category_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'primary_category_tokens')))
nuevo_registro_spark = nuevo_registro_spark.withColumn('product_name_cleaned', lemmatize_text_spark_udf('product_name_cleaned'))
nuevo_registro_spark = nuevo_registro_spark.withColumn('primary_category_cleaned', lemmatize_text_spark_udf('primary_category_cleaned'))

# Aplicar el mismo pipeline del modelo al nuevo registro
resultado_prediccion = loaded_model.transform(nuevo_registro_spark)

# Seleccionar las columnas relevantes del resultado de la predicción
resultado_seleccionado = resultado_prediccion.select('product_name', 'size', 'size_indexed', 'prediction', 'probability').show()



+--------------------+----+------------+----------+--------------------+
|        product_name|size|size_indexed|prediction|         probability|
+--------------------+----+------------+----------+--------------------+
|Dolex activgel 24...| XS3|         1.0|       1.0|[0.0,1.0,0.0,0.0,...|
+--------------------+----+------------+----------+--------------------+



#TERMINA TODO


# FIN

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType


# Crear un esquema para el DataFrame de Spark
schema = StructType([
    StructField("sellerName", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("shop_sku", StringType(), True),
    StructField("primary_category", StringType(), True),
    StructField("N1", StringType(), True),
    StructField("width_in_cm", FloatType(), True),
    StructField("length_in_cm", FloatType(), True),
    StructField("height_in_cm", FloatType(), True),
    StructField("weight_in_kg", FloatType(), True),
    StructField("volumen_cm3", FloatType(), True),
    StructField("size", StringType(), True),
])

# Inicializar la sesión de Spark
spark = SparkSession.builder.master("local").appName("SPM-SPARK").getOrCreate()

# Crear el DataFrame de Spark a partir de los nuevos datos
nuevos_datos_spark = spark.createDataFrame([nuevo_registro], schema=schema)

# Verificar las primeras filas del DataFrame de Spark con los nuevos datos
nuevos_datos_spark.show()


+------------+--------------+--------+--------------------+------------+-----------+------------+------------+------------+-----------+----+
|  sellerName|  product_name|shop_sku|    primary_category|          N1|width_in_cm|length_in_cm|height_in_cm|weight_in_kg|volumen_cm3|size|
+------------+--------------+--------+--------------------+------------+-----------+------------+------------+------------+-----------+----+
|Sneyder Shop|Refrigueradora|  SKU183|Electronomesticos...|Linea Blanca|     1000.0|      2400.0|       100.0|       300.0|      2.4E8|  EO|
+------------+--------------+--------+--------------------+------------+-----------+------------+------------+------------+-----------+----+



In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

2024-01-16 03:15:26.388724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 03:15:26.388790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 03:15:26.390736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 03:15:28.090898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 33.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import nltk
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import VectorAssembler


import nltk
nltk.download('stopwords')
nltk.download('punkt')


import spacy

# Cargar modelo de procesamiento de lenguaje natural en español
nlp = spacy.load('es_core_news_sm')

# Obtener las stopwords en español
spanish_stop_words = set(nltk.corpus.stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Verificar las primeras filas del DataFrame de Spark con los nuevos datos
nuevos_datos_spark.show()

# Imprimir el esquema del DataFrame
nuevos_datos_spark.printSchema()


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-41-39030096985e>", line 9, in lemmatize_text
  File "/usr/local/lib/python3.10/dist-packages/spacy/language.py", line 1030, in __call__
    doc = self._ensure_doc(text)
  File "/usr/local/lib/python3.10/dist-packages/spacy/language.py", line 1124, in _ensure_doc
    raise ValueError(Errors.E1041.format(type=type(doc_like)))
ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>


In [ ]:
# Definir funciones de tokenización y limpieza
def tokenize_and_clean(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in spanish_stop_words]
    return tokens

# Definir función de lematización
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Crear UDFs de Spark
tokenize_and_clean_spark_udf = udf(tokenize_and_clean_udf, ArrayType(StringType()))
lemmatize_text_spark_udf = udf(lemmatize_text_udf, StringType())

# Aplicar las transformaciones de texto en un solo paso
df_spark = df_spark.withColumn('product_name_tokens', tokenize_and_clean_spark_udf('product_name'))
df_spark = df_spark.withColumn('primary_category_tokens', tokenize_and_clean_spark_udf('primary_category'))
df_spark = df_spark.withColumn('product_name_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'product_name_tokens')))
df_spark = df_spark.withColumn('primary_category_cleaned', lemmatize_text_spark_udf(F.concat_ws(' ', 'primary_category_tokens')))

# TF-IDF en 'product_name' y 'primary_category'
# Asumiendo que 'loaded_model' y 'model_category' contienen los modelos que necesitas
nuevos_datos_spark = loaded_model.transform(nuevos_datos_spark)
nuevos_datos_spark = model_category.transform(nuevos_datos_spark)

# Reducción de palabras en las columnas lematizadas
nuevos_datos_spark = nuevos_datos_spark.withColumn('product_name_cleaned', F.expr("substring_index(product_name_cleaned, ' ', 2)"))
nuevos_datos_spark = nuevos_datos_spark.withColumn('primary_category_cleaned', F.expr("substring_index(primary_category_cleaned, ' ', 2)"))

# Asumiendo que las columnas de características son 'name_tfidf', 'category_tfidf', 'product_name_cleaned', y 'primary_category_cleaned'
feature_columns = ['name_tfidf', 'category_tfidf', 'product_name_cleaned', 'primary_category_cleaned']
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Ajustar columnas de características
nuevos_datos_spark = vector_assembler.transform(nuevos_datos_spark)

# Realizar predicciones
predicciones_nuevos_datos = loaded_model.transform(nuevos_datos_spark)

# Visualizar las predicciones
predicciones_nuevos_datos.select('prediction').show()

IllegalArgumentException: features does not exist. Available: sellerName, product_name, shop_sku, primary_category, N1, width_in_cm, length_in_cm, height_in_cm, weight_in_kg, volumen_cm3, size, product_name_tokens, primary_category_tokens, product_name_cleaned, primary_category_cleaned

# de aca abajo se mantiene

In [ ]:
# Crear un DataFrame de Spark con el nuevo registro
nuevos_datos = spark.createDataFrame([nuevo_registro], schema=schema)

# Tokenización y limpieza en 'product_name' y 'primary_category'
nuevos_datos = nuevos_datos.withColumn('product_name_tokens', tokenize_and_clean_spark_udf('product_name'))
nuevos_datos = nuevos_datos.withColumn('primary_category_tokens', tokenize_and_clean_spark_udf('primary_category'))

# Lematización en 'product_name' y 'primary_category'
nuevos_datos = nuevos_datos.withColumn('product_name_cleaned', lemmatize_text_spark_udf('product_name_tokens'))
nuevos_datos = nuevos_datos.withColumn('primary_category_cleaned', lemmatize_text_spark_udf('primary_category_tokens'))

# TF-IDF en 'product_name' y 'primary_category'
nuevos_datos = model_name.transform(nuevos_datos)
nuevos_datos = model_category.transform(nuevos_datos)

# Reducción de palabras en las columnas lematizadas
nuevos_datos = nuevos_datos.withColumn('product_name_cleaned', F.expr("substring_index(product_name_cleaned, ' ', 2)"))
nuevos_datos = nuevos_datos.withColumn('primary_category_cleaned', F.expr("substring_index(primary_category_cleaned, ' ', 2)"))

# Ajustar columnas de características
nuevos_datos_assembled = vector_assembler.transform(nuevos_datos).select('features')

# Realizar predicciones
predicciones_nuevos_datos = model.transform(nuevos_datos_assembled)

# Visualizar las predicciones
predicciones_nuevos_datos.select('prediction').show()


NameError: name 'tokenize_and_clean_spark_udf' is not defined

In [ ]:
# Crear un DataFrame de Spark con el nuevo registro
nuevos_datos = spark.createDataFrame([nuevo_registro], schema=schema)

# Tokenización y limpieza en 'product_name' y 'primary_category'
nuevos_datos = nuevos_datos.withColumn('product_name_tokens', tokenize_and_clean_spark_udf('product_name'))
nuevos_datos = nuevos_datos.withColumn('primary_category_tokens', tokenize_and_clean_spark_udf('primary_category'))

# Lematización en 'product_name' y 'primary_category'
nuevos_datos = nuevos_datos.withColumn('product_name_cleaned', lemmatize_text_spark_udf('product_name_tokens'))
nuevos_datos = nuevos_datos.withColumn('primary_category_cleaned', lemmatize_text_spark_udf('primary_category_tokens'))

# TF-IDF en 'product_name' y 'primary_category'
nuevos_datos = model_name.transform(nuevos_datos)
nuevos_datos = model_category.transform(nuevos_datos)

# Reducción de palabras en las columnas lematizadas
nuevos_datos = nuevos_datos.withColumn('product_name_cleaned', F.expr("substring_index(product_name_cleaned, ' ', 2)"))
nuevos_datos = nuevos_datos.withColumn('primary_category_cleaned', F.expr("substring_index(primary_category_cleaned, ' ', 2)"))

# Ajustar columnas de características
nuevos_datos_assembled = vector_assembler.transform(nuevos_datos).select('features')

# Realizar predicciones
predicciones_nuevos_datos = model.transform(nuevos_datos_assembled)

# Crear un DataFrame de mapeo entre prediction y Size
size_label_mapping = [
    (0.0, 'XS2'),
    (1.0, 'XS3'),
    (2.0, 'XS'),
    (3.0, 'M'),
    (4.0, 'S'),
    (5.0, 'L'),
    (6.0, 'EO'),
    (7.0, 'XL'),
    (8.0, 'LO'),
    (9.0, 'O'),
    (10.0, 'XXL')
]

size_label_mapping_df = spark.createDataFrame(size_label_mapping, ['label', 'Size'])

# Realizar join con el DataFrame de mapping
predicciones_nuevos_datos_with_size = predicciones_nuevos_datos.join(
    size_label_mapping_df,
    predicciones_nuevos_datos.prediction == size_label_mapping_df.label
)

# Seleccionar solo las columnas necesarias
result_df = predicciones_nuevos_datos_with_size.select('prediction', 'Size')

# Visualizar las predicciones con el tamaño correspondiente
result_df.show()
